<a href="https://colab.research.google.com/github/SuYenTing/104job_data_analyst_analysis/blob/main/%E7%8E%89%E5%B1%B1%E4%BA%BA%E5%B7%A5%E6%99%BA%E6%85%A7%E5%85%AC%E9%96%8B%E6%8C%91%E6%88%B0%E8%B3%BD2021%E5%86%AC%E5%AD%A3%E8%B3%BD%E7%A8%8B%E5%BC%8F%E7%A2%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 玉山人工智慧公開挑戰賽2021冬季賽 - 信用卡消費類別推薦
2022/01/09 蘇彥庭

* 競賽項目說明請參閱：[T-Brain官方網站-玉山人工智慧挑戰賽2021冬季賽](https://tbrain.trendmicro.com.tw/Competitions/Details/18)

* 競賽成果:
    * Public Leaderboard: 排名28/859，Top排名3.26%，分數：0.714550
    * Private Leaderboard: 排名22/859，Top排名3.26%，分數：0.713800

* 程式碼說明: 此程式碼是在Colab Pro環境下執行，由於本次競賽的資料集非常大(約3,000多萬筆資料)，若只單純用Colab跑會遇到記憶體不足的問題。我已有先減少特徵及訓練樣本數方便整理資料及模型訓練能夠順利，實際上在競賽時有用到更多特徵及訓練樣本，但模型訓練與預測的流程框架並沒有改變，相關的差異有註解在程式碼內提供參考。最後是在Azure上開一台虛擬主機，規格為一顆K80 GPU和56G記憶體才能順利訓練完整資料。

In [1]:
# 查看目前環境是否已設定好GPU
!nvidia-smi

Sun Jan  9 03:19:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# 掛載雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


由於資料非常大，所以此處利用Dask套件來協助處理資料。
* [Dask套件官方說明: https://dask.org/](https://dask.org/)

In [3]:
# 安裝dask套件
!python -m pip install "dask[complete]" -q

     |████████████████████████████████| 802 kB 9.0 MB/s 
     |████████████████████████████████| 132 kB 53.3 MB/s 
     |████████████████████████████████| 802 kB 49.4 MB/s 
     |████████████████████████████████| 793 kB 64.0 MB/s 
     |████████████████████████████████| 793 kB 59.4 MB/s 
     |████████████████████████████████| 791 kB 60.1 MB/s 
     |████████████████████████████████| 786 kB 67.2 MB/s 
     |████████████████████████████████| 779 kB 48.8 MB/s 
     |████████████████████████████████| 778 kB 60.4 MB/s 
     |████████████████████████████████| 776 kB 45.4 MB/s 
     |████████████████████████████████| 769 kB 48.1 MB/s 
     |████████████████████████████████| 766 kB 49.4 MB/s 
     |████████████████████████████████| 1.0 MB 47.4 MB/s 
     |████████████████████████████████| 722 kB 50.4 MB/s 
     |████████████████████████████████| 722 kB 47.3 MB/s 
     |████████████████████████████████| 715 kB 53.4 MB/s 
     |████████████████████████████████| 705 kB 49.2 MB/s 
     |█████████

In [4]:
# 由雲端硬碟複製及解壓縮官方提供的訓練資料集
!cp -r '/content/drive/MyDrive/2021_ESUN_winter/public_data.zip' 'public_data.zip'
!unzip -qq public_data.zip

In [1]:
# 載入套件
import os
import datetime
import pickle
import dask.dataframe as dd
import numpy as np
import pandas as pd
import plotly.express as px
import xgboost as xgb
import lightgbm as lgb
import math
import gc

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm_notebook as tqdm

In [2]:
# 將玉山提供的檔案轉為parquet格式
# 利用dask套件讀入資料
df = dd.read_csv('tbrain_cc_training_48tags_hash_final.csv', assume_missing=True)

# 以chid, dt排序 
df = df.set_index('chid')
df = df.map_partitions(lambda df: df.sort_values(['chid', 'dt']))

# 把csv切成32個parquet格式
df = df.repartition(npartitions=32) 
dd.to_parquet(df, path='data.parquet', engine='pyarrow')

In [3]:
# 利用dask套件讀取parquet資料
df = dd.read_parquet('data.parquet')

In [4]:
# 競賽預測目標類別
targetClass = ['2', '6', '10', '12', '13', '15', '18', '19', '21', '22', '25', '26', '36', '37', '39', '48']

In [5]:
# 整理出每個客戶最近期的輪廓資料
def makeCustomerProfileData():

    # 取出各chid最近期的客戶輪廓資訊
    iData = df.groupby(['chid'])[['dt']].max()
    iData = iData.merge(df[['dt', 'masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'slam', 'gender_code', 'age', 'primary_card']], how='left', on=['chid', 'dt'])
    iData = df.groupby(['chid']).first()
    iData = iData[['masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'slam', 'gender_code', 'age', 'primary_card']]
    iData = iData.compute()

    # 補足類別變數的缺值(以freq最高補足)
    categoryData = iData.drop(columns=['slam'])
    imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    categoryData = pd.DataFrame(imputer.fit_transform(categoryData), index=categoryData.index, columns=categoryData.columns)

    # 對連續變數做標準化 並將缺值補0
    continueData = iData[['slam']]
    # 過濾異常值
    outlierCt = continueData[['slam']].quantile([0.95]).to_numpy()[0][0]
    continueData.loc[continueData['slam'] >= outlierCt, ['slam']] = outlierCt
    # 進行標準化
    scaler = StandardScaler()
    continueData = pd.DataFrame(scaler.fit_transform(continueData[['slam']]), index=continueData.index, columns=continueData[['slam']].columns)    
    # 將缺值補0(即平均數)
    continueData = continueData.ffill(0)

    # 合併資料
    iData = pd.concat([categoryData, continueData], axis=1).reset_index()

    return iData

customerProfileData = makeCustomerProfileData()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## 整理naiveData

依據每個客戶過去24期各類別消費金額由高到低取前3名作為naive預測。
這份資料整理出來的目的是因為有發現部分客戶並不會在每一期都會消費，所以有可能發生該客戶沒有特徵資料的狀況讓我們能夠預測。
為彌補這樣的狀況，此時就透過Naive Data資料，來補足無法預測的部分。

In [6]:
# Naive預測: 以全期間各客戶類別消費金額由高到低做排序推薦
# 依據每個客戶過去24期各類別消費金額由高到低取前3名作為naive預測
# 篩選資料期間與預測目標
naiveData = df[df['shop_tag'].isin(targetClass)]
naiveData = naiveData.groupby(['chid', 'shop_tag'])['txn_amt'].sum()

# 執行DASK計算
naiveData = naiveData.compute()
naiveData = naiveData.reset_index()

# 依類別消費總金額結果由高到低排序
naiveData['rank'] = naiveData.groupby('chid')['txn_amt'].transform('rank', method='first', ascending=False)
naiveData['rank'] = naiveData['rank']+3  # 加3名 前3名留給模型預測用
naiveData = naiveData[['chid', 'shop_tag', 'rank']]

# 建立全體類別消費金額由高到低排序
allNaiveData = pd.DataFrame(data={'key': 1, 'rank': range(10, 17), 'shop_tag': ['37', '15', '36', '10', '2', '48', '12']})
# 建立完整客戶清單
chidData = pd.DataFrame(data={'key': 1, 'chid': df.index.drop_duplicates().compute()})
allNaiveData = allNaiveData.merge(chidData, on='key').drop(columns=['key'])

# 建立各客戶的預測推薦順序清單
naiveData = pd.concat([naiveData, allNaiveData])
naiveData = naiveData.groupby(['chid', 'shop_tag'])[['rank']].min().reset_index()  # 相同tag優先選取排序較前面的
naiveData = naiveData.sort_values(['chid', 'rank'])

In [7]:
# 整理各期消費金額資料整理放入dict內 方便待會整理特徵使用
featureDataDict = dict()
for dt in range(1, 25):

    # 篩選資料期間與預測目標
    featureData = df[df['dt'] == dt]
    featureData = featureData[featureData['shop_tag'].isin(targetClass)]
    featureData = featureData[['dt', 'shop_tag', 'txn_amt']]

    # 執行DASK計算
    featureData = featureData.compute()

    # 重設index
    featureData = featureData.reset_index()

    # 儲存資料
    featureDataDict[dt] = featureData

## 資料切割方式說明
* 訓練集資料：是以過去12期整理出的特徵資料，來預測下一期的類別金額排序。
會選12期是因為通常信用卡消費可能會有週期循環(例如年繳保費之類的)，故取12期。
* 測試集資料：以最後一期的特徵資料來預測下一期類別金額。

In [8]:
# 整理資料期數
# 此處減少訓練集樣本避免記憶體無法負荷
# dt_range = range(12, 25)  # 若記憶體充足 可整理更多資料做訓練樣本以提升模型準確度
dt_range = range(20, 25)

# 放入落後特徵期數
lagPeriodList = range(1, 12)

In [9]:
# 整理各期特徵資料
# 此處採用以下特徵
featureColumns = ['domestic_offline_cnt', 'domestic_online_cnt', 
                  'overseas_offline_cnt', 'overseas_online_cnt', 
                  'domestic_offline_amt_pct', 'domestic_online_amt_pct', 
                  'overseas_offline_amt_pct', 'overseas_online_amt_pct']

# 實際上我使用到以下特徵 但是若用這些特徵colab pro記憶體會無法負荷 所以後來改在雲端開VM主機(K80 GPU*1, 56GB記憶體)
# featureColumns = ['domestic_offline_cnt', 'domestic_online_cnt', 
#                   'overseas_offline_cnt', 'overseas_online_cnt',
#                   'domestic_offline_amt_pct', 'domestic_online_amt_pct',
#                   'overseas_offline_amt_pct', 'overseas_online_amt_pct',
#                   'card_1_txn_cnt', 'card_2_txn_cnt', 'card_3_txn_cnt', 'card_4_txn_cnt', 'card_5_txn_cnt',
#                   'card_6_txn_cnt', 'card_7_txn_cnt', 'card_8_txn_cnt', 'card_9_txn_cnt', 'card_10_txn_cnt',
#                   'card_11_txn_cnt', 'card_12_txn_cnt', 'card_13_txn_cnt', 'card_14_txn_cnt', 'card_other_txn_cnt',
#                   'card_1_txn_amt_pct', 'card_2_txn_amt_pct', 'card_3_txn_amt_pct',
#                   'card_4_txn_amt_pct', 'card_5_txn_amt_pct', 'card_6_txn_amt_pct',
#                   'card_7_txn_amt_pct', 'card_8_txn_amt_pct', 'card_9_txn_amt_pct',
#                   'card_10_txn_amt_pct', 'card_11_txn_amt_pct', 'card_12_txn_amt_pct',
#                   'card_13_txn_amt_pct', 'card_14_txn_amt_pct', 'card_other_txn_amt_pct']

cntFeatureDataDict = dict()
for dt in dt_range:
    featureData = df[df['dt'].isin(range(dt+1-12, dt+1))]
    featureData = featureData[featureData['shop_tag'].isin(targetClass)]
    featureData = featureData.groupby(['chid', 'shop_tag'])[featureColumns].sum()

    # 執行DASK計算
    featureData = featureData.compute()
    featureData = featureData.reset_index()

    # 儲存資料
    cntFeatureDataDict[dt] = featureData

## 預測目標金額處理方式

此次競賽主要是以NDCG@3作為評分指標，我們由官網提供的DCG定義來看：

$DCG_{c}=\sum_{i=1}^{i=3}\frac{V_{i,c}}{log_{2}(1+i)}$

分子的$V_{i,c}$為客戶在該類別消費的金額。

這邊要注意的是XGBoost的DCG定義為：

$DCG_{c}=\sum_{i=1}^{i=3}\frac{2^{rel_{i}}-1}{log_{2}(1+i)}$

其中$rel_{i}$為XGBoost Learing to Rank的預測目標。

由上可以發現在分子的部分定義是不一樣的，所以為能夠讓XGBoost的ndcg能夠符合競賽目標，所以此處我們會對預測目標做調整：

$V_{i,c} = 2^{rel_{i}}-1$

移項處理:

$rel_{i} = log_{2}(V_{i,c}+1)$

透過上式調整，即可讓競賽與XGBoost模型評分指標能夠一致。


In [10]:
# 整理模型訓練集與測試集資料
trainData = pd.DataFrame()
testData = pd.DataFrame()

for dt in tqdm(dt_range):

    # 建立chid與shop_tag完整清單
    iModelData = featureDataDict[dt][['chid', 'shop_tag']]
    for lag in lagPeriodList:
        iModelData = pd.concat([iModelData, featureDataDict[dt-lag][['chid', 'shop_tag']]])
    iModelData = iModelData.drop_duplicates()
    iModelData = iModelData.sort_values(['chid', 'shop_tag'])

    # 併入預測目標金額
    # dt=24為資料提供最後一期 故不需併入預測金額目標
    if dt < 24:
        iModelData = iModelData.merge(featureDataDict[dt+1][['chid', 'shop_tag', 'txn_amt']].rename(columns={'txn_amt': 'targetY'}), how='left', on=['chid', 'shop_tag'])
        iModelData = iModelData.fillna(0)
        # 整理預測目標排名(配合本次競賽ndcg) ref: https://en.wikipedia.org/wiki/Discounted_cumulative_gain
        iModelData['targetYRank'] = iModelData['targetY'].apply(lambda x: math.log2(x+1))

    # 加入dt欄位資訊
    iModelData.insert(0, 'dt', dt+1)

    # 整理對應的lag特徵資料
    for lag in lagPeriodList:
        lagData = featureDataDict[dt-lag]
        lagData = lagData.drop(columns=['dt'])
        lagData.columns = ['chid', 'shop_tag'] + [elem+'_lag_'+str(lag) for elem in lagData.columns if elem not in ['chid', 'shop_tag']]
        iModelData = iModelData.merge(lagData, how='left', on=['chid', 'shop_tag'])

    # 併入國內外消費狀況特徵資料
    iModelData = iModelData.merge(cntFeatureDataDict[dt], how='left', on=['chid', 'shop_tag'])

    # 將缺值資料欄位補0
    iModelData = iModelData.fillna(0)

    # 將shop_tag欄位轉為int
    iModelData['shop_tag'] = iModelData['shop_tag'].astype('int')

    # 建立qid(rank組別ID): 以各客戶各期不同消費類別的資料為同一組
    iModelData['qid'] = iModelData['chid']*100+iModelData['dt']
    iModelData['qid'] = iModelData['qid'].astype('int')

    # 併入客戶輪廓資料
    iModelData = iModelData.merge(customerProfileData, how='left', on=['chid'])

    # 併入模型資料
    if dt < 24:
        trainData = pd.concat([trainData, iModelData])
    elif dt == 24:
        testData = pd.concat([testData, iModelData])

  0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
# 刪除不需要的變數以節省記憶體空間
del featureDataDict
del customerProfileData
del cntFeatureDataDict
gc.collect()

68

## XGBoost模型訓練

* 為配合XGBoost模型的Learning to rank所需資料格式，在前面資料整理中，我們額外產生一個欄位qid，此qid將客戶在dt期消費中所有類別的樣本視為同一組
* 從訓練集資料中，依據qid隨機抽取10%做為驗證集資料，透過驗證集資料來找出模型最適合的最佳疊代數
* 此處礙於時間成本關係，僅找出最佳疊代數，並未找出最佳超參數組合，目前設定參數組合為手動調整出來的


In [12]:
# 分割訓練集及驗證集
qidList = pd.unique(trainData['qid']).tolist()
trainQid, validQid = train_test_split(qidList, test_size=0.1, random_state=666)
validData = trainData[trainData['qid'].isin(validQid)]
trainData = trainData[trainData['qid'].isin(trainQid)]

In [13]:
# 選取特徵
featureNames = trainData.columns.drop(['dt', 'targetY', 'targetYRank', 'qid'])

In [14]:
# 建立XGBoost矩陣
trainDMatrix = xgb.DMatrix(data=trainData[featureNames], label=trainData['targetYRank'])
trainDMatrix.set_group(trainData.groupby(['qid'])['qid'].count().tolist())  # 設定組別

validDMatrix = xgb.DMatrix(data=validData[featureNames], label=validData['targetYRank'])
validDMatrix.set_group(validData.groupby(['qid'])['qid'].count().tolist())  # 設定組別

testDMatrix = xgb.DMatrix(data=testData[featureNames])
testDMatrix.set_group(testData.groupby(['qid'])['qid'].count().tolist())  # 設定組別

In [15]:
# 訓練xgboost模型
watchlist = [(trainDMatrix, 'train'), (validDMatrix, 'valid')]
params = {
    'objective': 'rank:ndcg',
    'learning_rate': 0.3,  # 學習比率
    'max_depth': 6,  # 深度
    'colsample_bytree': 0.9,  # 隨機抽特徵(欄位)比率
    'subsample': 0.9,  # 隨機抽樣本比率
    'tree_method': 'gpu_hist',  # gpu學習
    'eval_metric': 'ndcg@3',
    'random_state': 666,
}
xgbModel = xgb.train(params=params, 
                     dtrain=trainDMatrix, 
                     num_boost_round=10000, 
                     evals=watchlist, 
                     early_stopping_rounds=20)

[0]	train-ndcg@3:0.806457	valid-ndcg@3:0.805946
Multiple eval metrics have been passed: 'valid-ndcg@3' will be used for early stopping.

Will train until valid-ndcg@3 hasn't improved in 20 rounds.
[1]	train-ndcg@3:0.808244	valid-ndcg@3:0.807852
[2]	train-ndcg@3:0.808961	valid-ndcg@3:0.808647
[3]	train-ndcg@3:0.809721	valid-ndcg@3:0.809284
[4]	train-ndcg@3:0.810281	valid-ndcg@3:0.809969
[5]	train-ndcg@3:0.810572	valid-ndcg@3:0.810339
[6]	train-ndcg@3:0.810767	valid-ndcg@3:0.810634
[7]	train-ndcg@3:0.811446	valid-ndcg@3:0.811212
[8]	train-ndcg@3:0.812028	valid-ndcg@3:0.811756
[9]	train-ndcg@3:0.812373	valid-ndcg@3:0.811939
[10]	train-ndcg@3:0.812762	valid-ndcg@3:0.812477
[11]	train-ndcg@3:0.813205	valid-ndcg@3:0.812825
[12]	train-ndcg@3:0.813567	valid-ndcg@3:0.813158
[13]	train-ndcg@3:0.813798	valid-ndcg@3:0.813276
[14]	train-ndcg@3:0.814001	valid-ndcg@3:0.81359
[15]	train-ndcg@3:0.8142	valid-ndcg@3:0.813787
[16]	train-ndcg@3:0.814425	valid-ndcg@3:0.81398
[17]	train-ndcg@3:0.814543	valid

In [16]:
# 紀錄模型預測結果
testData['predictY'] = xgbModel.predict(testDMatrix)
testData['rank'] = testData.groupby('chid')['predictY'].transform('rank', method='first', ascending=False)

# 將shop_tag欄位轉為int
testData['shop_tag'] = testData['shop_tag'].astype('str')

# 選取所需欄位
testData = testData[['chid', 'shop_tag', 'rank']]

In [17]:
# 選取每個客戶id前3名
predictResult = testData[testData['rank'] <= 3]

# 將naive預測加入模型預測清單
predictResult = pd.concat([predictResult, naiveData])

# 選取每個客戶rank前3名
predictResult = predictResult.groupby(['chid', 'shop_tag'])[['rank']].min().reset_index()  # 相同shop_tag優先選取排序較前面的
predictResult = predictResult.sort_values(['chid', 'rank'])
predictResult['rank'] = predictResult.groupby('chid')['rank'].transform('rank', method='first')
predictResult = predictResult[predictResult['rank'] <= 3]

# 轉為寬資料
submissionData = predictResult.pivot(values=['shop_tag'], index=['chid'], columns=['rank']).reset_index()
submissionData.columns = ['chid', 'top1', 'top2', 'top3']

print('缺值比率' + str(round(submissionData.isna().sum().sum()/(500000*3), 2)))

# 產出提交預測表
submissionData.to_csv(f'submissionData.csv', index=False)
submissionData.to_csv(f'/content/drive/MyDrive/2021_ESUN_winter/submissionData.csv', index=False)

缺值比率0.0


In [18]:
# 提交結果
submissionData

,chid,top1,top2,top3
0,10000000.0,10,48,12
1,10000001.0,48,12,37
2,10000002.0,37,22,10
3,10000003.0,37,22,15
4,10000004.0,12,37,36
...,...,...,...,...
499995,10499995.0,10,2,37
499996,10499996.0,36,15,48
499997,10499997.0,48,36,21
499998,10499998.0,10,39,37
